Dataset taken from: https://www.kaggle.com/datasets/imdevskp/corona-virus-report?resource=download

This proyect will be my first personal proyect on ML Regression. This proyect will help me better understand how Regression with multiple inputs works. This file will have errors and the Regression will not have the best evaluation.

Import all libraries needed

In [2]:
%tensorflow_version 2.x

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Upload the file downloaded on the computer to Collab

In [218]:
from google.colab import files
uploaded = files.upload()

Saving country_wise_latest.csv to country_wise_latest (1).csv


Read the file to a pandas dataframe and create a copy of it. Print the head of the dataframe to visualize the data present.

In [160]:
org_dataset = pd.read_csv('country_wise_latest.csv')
dataset = org_dataset.copy()
dataset.head()

/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


A very important part of ML is to understand what data you have on your hands. But another important part is cleaning the data. This data contains values that are infinte and nan. To properly clean this data we have to get rid of this unuseful data.

In [161]:
dataset.replace([np.inf, -np.inf], np.nan, inplace=True) #Turn infinites values to nan

In [162]:
dataset = dataset.dropna()

In [163]:
dataset.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In this approach I will eliminate most of the features and only stay with four: Confirmed, Deaths, Recovered and Active. Another approach is to transfrom Categorical Data into Numerical Data instead of getting rid of it.

In [166]:
dataset.drop(['WHO Region', 'Country/Region', '1 week % increase',
              'Deaths / 100 Cases', 'Deaths / 100 Cases', 'Recovered / 100 Cases',
              'Deaths / 100 Recovered', '1 week change', 'New cases',
              'New deaths', 'New recovered', 'Confirmed last week'], axis=1, inplace=True)
dataset.head()

,Confirmed,Deaths,Recovered,Active
0,36263,1269,25198,9796
1,4880,144,2745,1991
2,27973,1163,18837,7973
3,907,52,803,52
4,950,41,242,667


Split the dataset into the training dataset and testing dataset for the regression model.

In [168]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_dataset.head()

,Confirmed,Deaths,Recovered,Active
95,1219,31,1045,143
18,1770,35,1036,699
34,922,75,810,37
99,2827,64,577,2186
186,2704,36,542,2126


In [169]:
test_dataset.head()

,Confirmed,Deaths,Recovered,Active
9,20558,713,18246,1599
21,10498,294,4930,5274
25,10621,347,5585,4689
29,2328,22,1550,756
31,17110,391,14539,2180


View the shape of the datasets to get a better understanding of the different datasets present. This information will also be used to train the model.

In [170]:
print(dataset.shape , train_dataset.shape , test_dataset.shape)

(182, 4) (146, 4) (36, 4)


Split the features from the labels. Inspect the shape of the features.

In [171]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Deaths')
test_labels = test_features.pop('Deaths')

In [172]:
train_features.shape

(146, 3)

In [173]:
train_features.head()

,Confirmed,Recovered,Active
95,1219,1045,143
18,1770,1036,699
34,922,810,37
99,2827,577,2186
186,2704,542,2126


Visualize what are the mean and stadard deviation of the dataset to later normalize the data.

In [176]:
print(train_dataset.describe().transpose()[['mean', 'std']])

                   mean            std
Confirmed  98204.232877  430517.175428
Deaths      3647.630137   15340.765150
Recovered  57949.856164  213048.492835
Active     36606.746575  239930.187673


Normalize the data to use in the model. The normalizer will work as the input layer of the NN.

In [190]:
normalizer = keras.layers.Normalization(input_shape=(train_features.shape[1],), axis=-1)
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())

[[98204.234 57949.855 36606.746]]


/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [191]:
#Code extracted form tensorflow to visualize normalization
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[1219 1045  143]]

Normalized: [[-0.23 -0.27 -0.15]]


Create the model. This will be a five layer Dense Neural Network with a Rectified Linear Unit activation.

In [208]:
DNN_Model = keras.Sequential([
    normalizer, #Input Layer
    layers.Dense(64, activation='relu'), #Hidden Layer
    layers.Dense(64, activation='relu'), #Hidden Layer
    layers.Dense(64, activation='relu'), #Hidden Layer
    layers.Dense(1) #Output Layer
])

DNN_Model.compile(
    loss='mean_absolute_error',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1)
)

Get the summarty of the model to inspect its parameters.

In [202]:
DNN_Model.summary()

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ normalization_16 (Normalization)     │ (None, 3)                   │               7 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_140 (Dense)                    │ (None, 64)                  │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_141 (Dense)                    │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_142 (Dense)                    │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,488 (17.54 KB)

 Trainable params: 4,481 (17.50 KB)

 Non-trainable params: 7 (32.00 B)

Train the mopdel with an epochs of 50. I first trained it with 100 epchos but found that I lost more data. Also tried to train it with batch sizes but found that with out batches it had a better result.

In [213]:
history = DNN_Model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=1,
    epochs=50
)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1789.2551 - val_loss: 975.4639
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2139.8022 - val_loss: 891.2267
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1540.4846 - val_loss: 777.2918
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1469.3479 - val_loss: 796.0155
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1523.6764 - val_loss: 879.0165
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1402.2706 - val_loss: 906.6194
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1458.8179 - val_loss: 868.8152
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1495.7809 - val_loss: 924.6754
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1354.6331 - val_loss: 886.7362
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1407.2159 - val_loss: 900.9544
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1460.1045 - val_loss: 950.6157
Epoch 12/50
4/4 ━━━━━━━━━━━━━━

Finally evaluate the the model. I resulted with a high loss. My hypothesis is that this was because the mean of my dataset was exceptionally high (98000).

In [219]:
DNN_Model.evaluate(test_features, test_labels, verbose=1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1793.4634 


2112.856201171875